In [ ]:
### Notes to the graders

# Each of us pulled and cleaned data for 2 months of each top 200 and viral 50 charts --- the data wrangling parts are in separate notebooks for each of us. The code for the collection notebooks will not run without entering your own ID and secert ID for the spotify API. 

# We all worked on separate parts of the graphing and analysis and combined that into one jupyter notebook named "Graphing." This notebook is broken out into sections by name as well, but the code here is runable and the whole notebook is a compilation of how we approached creating our final graphs for the presentation.

# Created csv files are stored in the 'csv files' folder and created/saved graphs are stored in the 'Images' folder.

# Data Combining

#### CSV files from Spotify Charts were imported and merged.
Since the CSV files were per week, they were combined based on: https://stackoverflow.com/questions/42756696/read-multiple-csv-files-and-add-filename-as-new-column-in-pandas


In [ ]:
import datetime
import glob
import os
import pandas as pd
import json
import time
import numpy as np
import spotipy
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as st

### CSV combining for May and July

In [ ]:
files = glob.glob('Raw_files/top_200/may/*.csv')
# print(files)

may_combined_df_top200 = pd.concat([pd.read_csv(fp, skiprows=[0], skip_blank_lines=True).assign(Filename=os.path.basename(fp)) for fp in files])

may_combined_df_top200
may_combined_df_top200 = may_combined_df_top200.sort_values(['Filename', "Position"], ascending=True)

In [ ]:
may_combined_df_top200["Month"] = ""
may_combined_df_top200['Date'] = may_combined_df_top200['Filename'].str.slice(19,29)
may_combined_df_top200['Date'] = pd.to_datetime(may_combined_df_top200['Date'])
may_combined_df_top200["Week Number"] = may_combined_df_top200['Date'].dt.week
may_combined_df_top200['Type'] = ''


for index, row in may_combined_df_top200.iterrows():
    may_combined_df_top200.loc[index, "Month"] = "May"
    may_combined_df_top200.loc[index, "Type"] = "Top 200"

#viral slicing is 16, 26
#top 200 slicing is 19,29
may_combined_df_top200.to_csv("csv files/may_combined_top200.csv")



may_combined_df_top200

In [ ]:
files = glob.glob('Raw_files/viral/may/*.csv')
# print(files)

may_combined_df_viral50 = pd.concat([pd.read_csv(fp).assign(Filename=os.path.basename(fp)) for fp in files])

may_combined_df_viral50
may_combined_df_viral50 = may_combined_df_viral50.sort_values(['Filename', "Position"], ascending=True)

In [ ]:
may_combined_df_viral50["Month"] = ""
may_combined_df_viral50['Date'] = may_combined_df_viral50['Filename'].str.slice(16,26)
may_combined_df_viral50['Date'] = pd.to_datetime(may_combined_df_viral50['Date'])
may_combined_df_viral50["Week Number"] = may_combined_df_viral50['Date'].dt.week
may_combined_df_viral50['Type'] = ''


for index, row in may_combined_df_viral50.iterrows():
    may_combined_df_viral50.loc[index, "Month"] = "May"
    may_combined_df_viral50.loc[index, "Type"] = "Viral 50"

#viral slicing is 16, 26
#top 200 slicing is 19,29
may_combined_df_viral50.to_csv("csv files/may_combined_viral50.csv")



may_combined_df_viral50

In [ ]:
files = glob.glob('Raw_files/top_200/jul/*.csv')
# print(files)

july_combined_df_top200 = pd.concat([pd.read_csv(fp, skiprows=[0], skip_blank_lines=True).assign(Filename=os.path.basename(fp)) for fp in files])

july_combined_df_top200
july_combined_df_top200 = july_combined_df_top200.sort_values(['Filename', "Position"], ascending=True)

In [ ]:
july_combined_df_top200["Month"] = ""
july_combined_df_top200['Date'] = july_combined_df_top200['Filename'].str.slice(19,29)
july_combined_df_top200['Date'] = pd.to_datetime(july_combined_df_top200['Date'])
july_combined_df_top200["Week Number"] = july_combined_df_top200['Date'].dt.week
july_combined_df_top200['Type'] = ''


for index, row in july_combined_df_top200.iterrows():
    july_combined_df_top200.loc[index, "Month"] = "July"
    july_combined_df_top200.loc[index, "Type"] = "Top 200"

#viral slicing is 16, 26
#top 200 slicing is 19,29
july_combined_df_top200.to_csv("csv files/july_combined_top200.csv")



july_combined_df_top200

In [ ]:
files = glob.glob('Raw_files/viral/jul/*.csv')
# print(files)

july_combined_df_viral50 = pd.concat([pd.read_csv(fp).assign(Filename=os.path.basename(fp)) for fp in files])

july_combined_df_viral50
july_combined_df_viral50 = july_combined_df_viral50.sort_values(['Filename', "Position"], ascending=True)

In [ ]:
july_combined_df_viral50["Month"] = ""
july_combined_df_viral50['Date'] = july_combined_df_viral50['Filename'].str.slice(16,26)
july_combined_df_viral50['Date'] = pd.to_datetime(july_combined_df_viral50['Date'])
july_combined_df_viral50["Week Number"] = july_combined_df_viral50['Date'].dt.week
july_combined_df_viral50['Type'] = ''


for index, row in july_combined_df_viral50.iterrows():
    july_combined_df_viral50.loc[index, "Month"] = "July"
    july_combined_df_viral50.loc[index, "Type"] = "Viral 50"

#viral slicing is 16, 26
#top 200 slicing is 19,29
july_combined_df_viral50.to_csv("csv files/july_combined_viral50.csv")



july_combined_df_viral50

# Preparing data for graphs
### Creating dataframes for the necessary information.

The data obtained from the Spotify Charts website, saved as csv and combined into months is used to make requests to Spotify API (using spotipy Python library) to get artist, track, and genre. This information is joined with the stream numbers information from the Spotify Charts csv, cleaned and get ready for graphing.

In [ ]:
#Spotify credentials
client_id = 'client_id'
client_secret = 'client_secret'

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)'

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

### <u>Viral 50</u>

### January

In [ ]:
csv_path = 'jan_viral_50.csv'
jan_viral_df = pd.read_csv(csv_path)
jan_viral_df.head()
jan_viral_df['Artist ID'] = ""
jan_viral_df['Genre 1'] = ""
jan_viral_df['Genre 2'] = ""
jan_viral_df['Genre 3'] = ""
# Make empty list to collect the artist ID
artist_id = []

for index, row in jan_viral_df.iterrows():

    track_id = row['URL']
    track_features = sp.track(track_id)
    # print(track_features)
    # viral_df.loc[index, 'Artist ID'] 
    # artist_id.append(track_features['album']['artists'][0]['id'])
    jan_viral_df.loc[index, 'Artist ID'] = track_features['album']['artists'][0]['id']

for index, row in jan_viral_df.iterrows():

    genre_id = row['Artist ID']
    artist_features = sp.artist(genre_id)
    try:
        jan_viral_df.loc[index, 'Genre 1'] = artist_features['genres'][0]
        jan_viral_df.loc[index, 'Genre 2'] = artist_features['genres'][1]
        jan_viral_df.loc[index, 'Genre 3'] = artist_features['genres'][2]
    except (IndexError, KeyError):
        print(f'Index {index} genre not found... skipped')
    
    time.sleep(0.1)
jan_viral_df.head()

In [ ]:
new_jan_viral_df=jan_viral_df.copy()
for index, row in new_jan_viral_df.iterrows():
    if row['Genre 1'] == "" :
        new_jan_viral_df.loc[index, 'Genre 1'] = "unspecified"

new_jan_viral_df.value_counts('Genre 1')

In [ ]:
new_jan_viral_df_dropped = new_jan_viral_df.drop(["Unnamed: 0"], axis=1)
new_jan_viral_df_dropped.head()

### March

In [ ]:
csv_path = 'mar_viral_50.csv'
mar_viral_df = pd.read_csv(csv_path)
mar_viral_df.head()
mar_viral_df['Artist ID'] = ""
mar_viral_df['Genre 1'] = ""
mar_viral_df['Genre 2'] = ""
mar_viral_df['Genre 3'] = ""
# Make empty list to collect the artist ID
artist_id = []

for index, row in mar_viral_df.iterrows():

    track_id = row['URL']
    track_features = sp.track(track_id)
    # print(track_features)
    # viral_df.loc[index, 'Artist ID'] 
    # artist_id.append(track_features['album']['artists'][0]['id'])
    mar_viral_df.loc[index, 'Artist ID'] = track_features['album']['artists'][0]['id']

for index, row in mar_viral_df.iterrows():

    genre_id = row['Artist ID']
    artist_features = sp.artist(genre_id)
    try:
        mar_viral_df.loc[index, 'Genre 1'] = artist_features['genres'][0]
        mar_viral_df.loc[index, 'Genre 2'] = artist_features['genres'][1]
        mar_viral_df.loc[index, 'Genre 3'] = artist_features['genres'][2]
    except (IndexError, KeyError):
        print(f'Index {index} genre not found... skipped')
    
    time.sleep(0.1)
mar_viral_df.head()

In [ ]:
new_mar_viral_df=mar_viral_df.copy()
for index, row in new_mar_viral_df.iterrows():
    if row['Genre 1'] == "" :
        new_mar_viral_df.loc[index, 'Genre 1'] = "unspecified"

new_mar_viral_df.value_counts('Genre 1')

### May

In [ ]:
csv_path = 'may_combined_df_viral50.csv'
may_viral_df = pd.read_csv(csv_path)
may_viral_df.head()

# Make blank column for storing the artist ID
may_viral_df['Artist ID'] = ""
may_viral_df['Genre 1'] = ""
may_viral_df['Genre 2'] = ""
may_viral_df['Genre 3'] = ""
# Make empty list to collect the artist ID
artist_id = []

for index, row in may_viral_df.iterrows():

    track_id = row['URL']
    track_features = sp.track(track_id)
    # print(track_features)
    # viral_df.loc[index, 'Artist ID'] 
    # artist_id.append(track_features['album']['artists'][0]['id'])
    may_viral_df.loc[index, 'Artist ID'] = track_features['album']['artists'][0]['id']

for index, row in may_viral_df.iterrows():

    genre_id = row['Artist ID']
    artist_features = sp.artist(genre_id)
    try:
        may_viral_df.loc[index, 'Genre 1'] = artist_features['genres'][0]
        may_viral_df.loc[index, 'Genre 2'] = artist_features['genres'][1]
        may_viral_df.loc[index, 'Genre 3'] = artist_features['genres'][2]
    except (IndexError, KeyError):
        print(f'Index {index} genre not found... skipped')
    
    time.sleep(0.1)

In [ ]:
new_may_viral_df=may_viral_df.copy()
for index, row in new_may_viral_df.iterrows():
    if row['Genre 1'] == "" :
        new_may_viral_df.loc[index, 'Genre 1'] = "unspecified"

new_may_viral_df.value_counts('Genre 1')

In [ ]:
new_may_viral_df_dropped = new_may_viral_df.drop(["Unnamed: 0"], axis=1)
new_may_viral_df_dropped.head()

### July

In [ ]:
csv_path = 'july_combined_df_viral50.csv'
jul_viral_df = pd.read_csv(csv_path)
jul_viral_df.head()

# Make blank column for storing the artist ID
jul_viral_df['Artist ID'] = ""
jul_viral_df['Genre 1'] = ""
jul_viral_df['Genre 2'] = ""
jul_viral_df['Genre 3'] = ""
# Make empty list to collect the artist ID
artist_id = []

for index, row in jul_viral_df.iterrows():

    track_id = row['URL']
    track_features = sp.track(track_id)
    # print(track_features)
    # viral_df.loc[index, 'Artist ID'] 
    # artist_id.append(track_features['album']['artists'][0]['id'])
    jul_viral_df.loc[index, 'Artist ID'] = track_features['album']['artists'][0]['id']

for index, row in jul_viral_df.iterrows():

    genre_id = row['Artist ID']
    artist_features = sp.artist(genre_id)
    try:
        jul_viral_df.loc[index, 'Genre 1'] = artist_features['genres'][0]
        jul_viral_df.loc[index, 'Genre 2'] = artist_features['genres'][1]
        jul_viral_df.loc[index, 'Genre 3'] = artist_features['genres'][2]
    except (IndexError, KeyError):
        print(f'Index {index} genre not found... skipped')
    
    time.sleep(0.1)

In [ ]:
new_jul_viral_df=jul_viral_df.copy()
for index, row in new_jul_viral_df.iterrows():
    if row['Genre 1'] == "" :
        new_jul_viral_df.loc[index, 'Genre 1'] = "unspecified"

new_jul_viral_df.value_counts('Genre 1')

In [ ]:
new_jul_viral_df_dropped = new_jul_viral_df.drop(["Unnamed: 0"], axis=1)
new_jul_viral_df_dropped.head()

### September

In [ ]:
csv_path = 'sept_combined_viral.csv'
sep_viral_df = pd.read_csv(csv_path)
sep_viral_df.head()

# Make blank column for storing the artist ID
sep_viral_df['Artist ID'] = ""
sep_viral_df['Genre 1'] = ""
sep_viral_df['Genre 2'] = ""
sep_viral_df['Genre 3'] = ""
# Make empty list to collect the artist ID
artist_id = []

for index, row in sep_viral_df.iterrows():

    track_id = row['URL']
    track_features = sp.track(track_id)
    # print(track_features)
    # viral_df.loc[index, 'Artist ID'] 
    # artist_id.append(track_features['album']['artists'][0]['id'])
    sep_viral_df.loc[index, 'Artist ID'] = track_features['album']['artists'][0]['id']

for index, row in sep_viral_df.iterrows():

    genre_id = row['Artist ID']
    artist_features = sp.artist(genre_id)
    try:
        sep_viral_df.loc[index, 'Genre 1'] = artist_features['genres'][0]
        sep_viral_df.loc[index, 'Genre 2'] = artist_features['genres'][1]
        sep_viral_df.loc[index, 'Genre 3'] = artist_features['genres'][2]
    except (IndexError, KeyError):
        print(f'Index {index} genre not found... skipped')
    
    time.sleep(0.1)

In [ ]:
new_sep_viral_df=sep_viral_df.copy()
for index, row in new_sep_viral_df.iterrows():
    if row['Genre 1'] == "" :
        new_sep_viral_df.loc[index, 'Genre 1'] = "unspecified"

new_sep_viral_df.value_counts('Genre 1')

In [ ]:
new_sep_viral_df_dropped = new_sep_viral_df.drop(["Unnamed: 0"], axis=1)
new_sep_viral_df_dropped.head()

### November

In [ ]:
csv_path = 'nov_combined_viral.csv'
nov_viral_df = pd.read_csv(csv_path)
nov_viral_df.head()

# Make blank column for storing the artist ID
nov_viral_df['Artist ID'] = ""
nov_viral_df['Genre 1'] = ""
nov_viral_df['Genre 2'] = ""
nov_viral_df['Genre 3'] = ""
# Make empty list to collect the artist ID
artist_id = []

for index, row in nov_viral_df.iterrows():

    track_id = row['URL']
    track_features = sp.track(track_id)
    # print(track_features)
    # viral_df.loc[index, 'Artist ID'] 
    # artist_id.append(track_features['album']['artists'][0]['id'])
    nov_viral_df.loc[index, 'Artist ID'] = track_features['album']['artists'][0]['id']

for index, row in nov_viral_df.iterrows():

    genre_id = row['Artist ID']
    artist_features = sp.artist(genre_id)
    try:
        nov_viral_df.loc[index, 'Genre 1'] = artist_features['genres'][0]
        nov_viral_df.loc[index, 'Genre 2'] = artist_features['genres'][1]
        nov_viral_df.loc[index, 'Genre 3'] = artist_features['genres'][2]
    except (IndexError, KeyError):
        print(f'Index {index} genre not found... skipped')
    
    time.sleep(0.1)

In [ ]:
new_nov_viral_df=nov_viral_df.copy()
for index, row in new_nov_viral_df.iterrows():
    if row['Genre 1'] == "" :
        new_nov_viral_df.loc[index, 'Genre 1'] = "unspecified"

new_nov_viral_df.value_counts('Genre 1')

In [ ]:
new_nov_viral_df_dropped = new_nov_viral_df.drop(["Unnamed: 0"], axis=1)
new_nov_viral_df_dropped.head()

Combining all dataframes and export to CSV to be used for total/all months information.

In [ ]:
#combine dataframes and reindex
viral_all_df_reindex = pd.concat([new_jan_viral_df_dropped, new_mar_viral_df_dropped, new_may_viral_df_dropped,new_jul_viral_df_dropped, new_sep_viral_df_dropped, new_nov_viral_df_dropped], ignore_index=True)
viral_all_df_reindex

In [ ]:
#save as csv
viral_all_df_reindex.to_csv('viral_all.csv')

## Obtaining genre data from dataframes

### <u>Viral 50</u>

### January

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('rap|hip hop|trap') | new_jan_viral_df['Genre 2'].str.contains('rap|hip hop|trap') | new_jan_viral_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('r&b|soul|blues') | new_jan_viral_df['Genre 2'].str.contains('r&b|soul|blues') | new_jan_viral_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('pop') | new_jan_viral_df['Genre 2'].str.contains('pop') | new_jan_viral_df['Genre 3'].str.contains('pop')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('rock') | new_jan_viral_df['Genre 2'].str.contains('rock') | new_jan_viral_df['Genre 3'].str.contains('rock')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('country') | new_jan_viral_df['Genre 2'].str.contains('country') | new_jan_viral_df['Genre 3'].str.contains('country')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('latin') | new_jan_viral_df['Genre 2'].str.contains('latin') | new_jan_viral_df['Genre 3'].str.contains('latin')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('disco') | new_jan_viral_df['Genre 2'].str.contains('disco') | new_jan_viral_df['Genre 3'].str.contains('disco')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('indie') | new_jan_viral_df['Genre 2'].str.contains('indie') | new_jan_viral_df['Genre 3'].str.contains('indie')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('house|edm') | new_jan_viral_df['Genre 2'].str.contains('house|edm') | new_jan_viral_df['Genre 3'].str.contains('house|edm')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('alt|alternative') | new_jan_viral_df['Genre 2'].str.contains('alt|alternative') | new_jan_viral_df['Genre 3'].str.contains('alt|alternative')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('glitchcore') | new_jan_viral_df['Genre 2'].str.contains('glitchcore') | new_jan_viral_df['Genre 3'].str.contains('glitchcore')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('brooklyn drill') | new_jan_viral_df['Genre 2'].str.contains('brooklyn drill') | new_jan_viral_df['Genre 3'].str.contains('brooklyn drill')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('halloween') | new_jan_viral_df['Genre 2'].str.contains('halloween') | new_jan_viral_df['Genre 3'].str.contains('halloween')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('christmas') | new_jan_viral_df['Genre 2'].str.contains('christmas') | new_jan_viral_df['Genre 3'].str.contains('christmas')]), len(new_jan_viral_df[new_jan_viral_df['Genre 1'].str.contains('classical') | new_jan_viral_df['Genre 2'].str.contains('classical') | new_jan_viral_df['Genre 3'].str.contains('classical')])]
}
jan_genre_df = pd.DataFrame(genre_data)
jan_genre_df.head()

### March

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('rap|hip hop|trap') | new_mar_viral_df['Genre 2'].str.contains('rap|hip hop|trap') | new_mar_viral_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('r&b|soul|blues') | new_mar_viral_df['Genre 2'].str.contains('r&b|soul|blues') | new_mar_viral_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('pop') | new_mar_viral_df['Genre 2'].str.contains('pop') | new_mar_viral_df['Genre 3'].str.contains('pop')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('rock') | new_mar_viral_df['Genre 2'].str.contains('rock') | new_mar_viral_df['Genre 3'].str.contains('rock')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('country') | new_mar_viral_df['Genre 2'].str.contains('country') | new_mar_viral_df['Genre 3'].str.contains('country')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('latin') | new_mar_viral_df['Genre 2'].str.contains('latin') | new_mar_viral_df['Genre 3'].str.contains('latin')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('disco') | new_mar_viral_df['Genre 2'].str.contains('disco') | new_mar_viral_df['Genre 3'].str.contains('disco')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('indie') | new_mar_viral_df['Genre 2'].str.contains('indie') | new_mar_viral_df['Genre 3'].str.contains('indie')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('house|edm') | new_mar_viral_df['Genre 2'].str.contains('house|edm') | new_mar_viral_df['Genre 3'].str.contains('house|edm')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('alt|alternative') | new_mar_viral_df['Genre 2'].str.contains('alt|alternative') | new_mar_viral_df['Genre 3'].str.contains('alt|alternative')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('glitchcore') | new_mar_viral_df['Genre 2'].str.contains('glitchcore') | new_mar_viral_df['Genre 3'].str.contains('glitchcore')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('brooklyn drill') | new_mar_viral_df['Genre 2'].str.contains('brooklyn drill') | new_mar_viral_df['Genre 3'].str.contains('brooklyn drill')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('halloween') | new_mar_viral_df['Genre 2'].str.contains('halloween') | new_mar_viral_df['Genre 3'].str.contains('halloween')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('christmas') | new_mar_viral_df['Genre 2'].str.contains('christmas') | new_mar_viral_df['Genre 3'].str.contains('christmas')]), len(new_mar_viral_df[new_mar_viral_df['Genre 1'].str.contains('classical') | new_mar_viral_df['Genre 2'].str.contains('classical') | new_mar_viral_df['Genre 3'].str.contains('classical')])]
}
mar_genre_df = pd.DataFrame(genre_data)
mar_genre_df.head()

### May

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('rap|hip hop|trap') | new_may_viral_df['Genre 2'].str.contains('rap|hip hop|trap') | new_may_viral_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('r&b|soul|blues') | new_may_viral_df['Genre 2'].str.contains('r&b|soul|blues') | new_may_viral_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('pop') | new_may_viral_df['Genre 2'].str.contains('pop') | new_may_viral_df['Genre 3'].str.contains('pop')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('rock') | new_may_viral_df['Genre 2'].str.contains('rock') | new_may_viral_df['Genre 3'].str.contains('rock')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('country') | new_may_viral_df['Genre 2'].str.contains('country') | new_may_viral_df['Genre 3'].str.contains('country')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('latin') | new_may_viral_df['Genre 2'].str.contains('latin') | new_may_viral_df['Genre 3'].str.contains('latin')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('disco') | new_may_viral_df['Genre 2'].str.contains('disco') | new_may_viral_df['Genre 3'].str.contains('disco')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('indie') | new_may_viral_df['Genre 2'].str.contains('indie') | new_may_viral_df['Genre 3'].str.contains('indie')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('house|edm') | new_may_viral_df['Genre 2'].str.contains('house|edm') | new_may_viral_df['Genre 3'].str.contains('house|edm')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('alt|alternative') | new_may_viral_df['Genre 2'].str.contains('alt|alternative') | new_may_viral_df['Genre 3'].str.contains('alt|alternative')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('glitchcore') | new_may_viral_df['Genre 2'].str.contains('glitchcore') | new_may_viral_df['Genre 3'].str.contains('glitchcore')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('brooklyn drill') | new_may_viral_df['Genre 2'].str.contains('brooklyn drill') | new_may_viral_df['Genre 3'].str.contains('brooklyn drill')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('halloween') | new_may_viral_df['Genre 2'].str.contains('halloween') | new_may_viral_df['Genre 3'].str.contains('halloween')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('christmas') | new_may_viral_df['Genre 2'].str.contains('christmas') | new_may_viral_df['Genre 3'].str.contains('christmas')]), len(new_may_viral_df[new_may_viral_df['Genre 1'].str.contains('classical') | new_may_viral_df['Genre 2'].str.contains('classical') | new_may_viral_df['Genre 3'].str.contains('classical')])]
}
may_genre_df = pd.DataFrame(genre_data)
may_genre_df.head()

### July

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('rap|hip hop|trap') | new_jul_viral_df['Genre 2'].str.contains('rap|hip hop|trap') | new_jul_viral_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('r&b|soul|blues') | new_jul_viral_df['Genre 2'].str.contains('r&b|soul|blues') | new_jul_viral_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('pop') | new_jul_viral_df['Genre 2'].str.contains('pop') | new_jul_viral_df['Genre 3'].str.contains('pop')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('rock') | new_jul_viral_df['Genre 2'].str.contains('rock') | new_jul_viral_df['Genre 3'].str.contains('rock')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('country') | new_jul_viral_df['Genre 2'].str.contains('country') | new_jul_viral_df['Genre 3'].str.contains('country')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('latin') | new_jul_viral_df['Genre 2'].str.contains('latin') | new_jul_viral_df['Genre 3'].str.contains('latin')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('disco') | new_jul_viral_df['Genre 2'].str.contains('disco') | new_jul_viral_df['Genre 3'].str.contains('disco')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('indie') | new_jul_viral_df['Genre 2'].str.contains('indie') | new_jul_viral_df['Genre 3'].str.contains('indie')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('house|edm') | new_jul_viral_df['Genre 2'].str.contains('house|edm') | new_jul_viral_df['Genre 3'].str.contains('house|edm')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('alt|alternative') | new_jul_viral_df['Genre 2'].str.contains('alt|alternative') | new_jul_viral_df['Genre 3'].str.contains('alt|alternative')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('glitchcore') | new_jul_viral_df['Genre 2'].str.contains('glitchcore') | new_jul_viral_df['Genre 3'].str.contains('glitchcore')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('brooklyn drill') | new_jul_viral_df['Genre 2'].str.contains('brooklyn drill') | new_jul_viral_df['Genre 3'].str.contains('brooklyn drill')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('halloween') | new_jul_viral_df['Genre 2'].str.contains('halloween') | new_jul_viral_df['Genre 3'].str.contains('halloween')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('christmas') | new_jul_viral_df['Genre 2'].str.contains('christmas') | new_jul_viral_df['Genre 3'].str.contains('christmas')]), len(new_jul_viral_df[new_jul_viral_df['Genre 1'].str.contains('classical') | new_jul_viral_df['Genre 2'].str.contains('classical') | new_jul_viral_df['Genre 3'].str.contains('classical')])]
}
jul_genre_df = pd.DataFrame(genre_data)
jul_genre_df.head()

### September

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('rap|hip hop|trap') | new_sep_viral_df['Genre 2'].str.contains('rap|hip hop|trap') | new_sep_viral_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('r&b|soul|blues') | new_sep_viral_df['Genre 2'].str.contains('r&b|soul|blues') | new_sep_viral_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('pop') | new_sep_viral_df['Genre 2'].str.contains('pop') | new_sep_viral_df['Genre 3'].str.contains('pop')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('rock') | new_sep_viral_df['Genre 2'].str.contains('rock') | new_sep_viral_df['Genre 3'].str.contains('rock')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('country') | new_sep_viral_df['Genre 2'].str.contains('country') | new_sep_viral_df['Genre 3'].str.contains('country')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('latin') | new_sep_viral_df['Genre 2'].str.contains('latin') | new_sep_viral_df['Genre 3'].str.contains('latin')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('disco') | new_sep_viral_df['Genre 2'].str.contains('disco') | new_sep_viral_df['Genre 3'].str.contains('disco')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('indie') | new_sep_viral_df['Genre 2'].str.contains('indie') | new_sep_viral_df['Genre 3'].str.contains('indie')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('house|edm') | new_sep_viral_df['Genre 2'].str.contains('house|edm') | new_sep_viral_df['Genre 3'].str.contains('house|edm')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('alt|alternative') | new_sep_viral_df['Genre 2'].str.contains('alt|alternative') | new_sep_viral_df['Genre 3'].str.contains('alt|alternative')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('glitchcore') | new_sep_viral_df['Genre 2'].str.contains('glitchcore') | new_sep_viral_df['Genre 3'].str.contains('glitchcore')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('brooklyn drill') | new_sep_viral_df['Genre 2'].str.contains('brooklyn drill') | new_sep_viral_df['Genre 3'].str.contains('brooklyn drill')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('halloween') | new_sep_viral_df['Genre 2'].str.contains('halloween') | new_sep_viral_df['Genre 3'].str.contains('halloween')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('christmas') | new_sep_viral_df['Genre 2'].str.contains('christmas') | new_sep_viral_df['Genre 3'].str.contains('christmas')]), len(new_sep_viral_df[new_sep_viral_df['Genre 1'].str.contains('classical') | new_sep_viral_df['Genre 2'].str.contains('classical') | new_sep_viral_df['Genre 3'].str.contains('classical')])]
}
sep_genre_df = pd.DataFrame(genre_data)
sep_genre_df.head()

### November

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('rap|hip hop|trap') | new_nov_viral_df['Genre 2'].str.contains('rap|hip hop|trap') | new_nov_viral_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('r&b|soul|blues') | new_nov_viral_df['Genre 2'].str.contains('r&b|soul|blues') | new_nov_viral_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('pop') | new_nov_viral_df['Genre 2'].str.contains('pop') | new_nov_viral_df['Genre 3'].str.contains('pop')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('rock') | new_nov_viral_df['Genre 2'].str.contains('rock') | new_nov_viral_df['Genre 3'].str.contains('rock')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('country') | new_nov_viral_df['Genre 2'].str.contains('country') | new_nov_viral_df['Genre 3'].str.contains('country')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('latin') | new_nov_viral_df['Genre 2'].str.contains('latin') | new_nov_viral_df['Genre 3'].str.contains('latin')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('disco') | new_nov_viral_df['Genre 2'].str.contains('disco') | new_nov_viral_df['Genre 3'].str.contains('disco')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('indie') | new_nov_viral_df['Genre 2'].str.contains('indie') | new_nov_viral_df['Genre 3'].str.contains('indie')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('house|edm') | new_nov_viral_df['Genre 2'].str.contains('house|edm') | new_nov_viral_df['Genre 3'].str.contains('house|edm')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('alt|alternative') | new_nov_viral_df['Genre 2'].str.contains('alt|alternative') | new_nov_viral_df['Genre 3'].str.contains('alt|alternative')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('glitchcore') | new_nov_viral_df['Genre 2'].str.contains('glitchcore') | new_nov_viral_df['Genre 3'].str.contains('glitchcore')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('brooklyn drill') | new_nov_viral_df['Genre 2'].str.contains('brooklyn drill') | new_nov_viral_df['Genre 3'].str.contains('brooklyn drill')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('halloween') | new_nov_viral_df['Genre 2'].str.contains('halloween') | new_nov_viral_df['Genre 3'].str.contains('halloween')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('christmas') | new_nov_viral_df['Genre 2'].str.contains('christmas') | new_nov_viral_df['Genre 3'].str.contains('christmas')]), len(new_nov_viral_df[new_nov_viral_df['Genre 1'].str.contains('classical') | new_nov_viral_df['Genre 2'].str.contains('classical') | new_nov_viral_df['Genre 3'].str.contains('classical')])]
}
nov_genre_df = pd.DataFrame(genre_data)
nov_genre_df.head()

## Visualizations: Histograms/Bar Graphs for Top Viral Songs

Create and clean dataframes. Plethora of odd genres unified into more general categories (original genre name from API reques was used as guide)

In [ ]:
viral_all_df = pd.read_csv('viral_all.csv')
viral_all_df.head()

In [ ]:
new_viral_all_df=viral_all_df.drop(["Unnamed: 0"], axis=1)
new_viral_all_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('rap|hip hop|trap') | new_viral_all_df['Genre 2'].str.contains('rap|hip hop|trap') | new_viral_all_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('r&b|soul|blues') | new_viral_all_df['Genre 2'].str.contains('r&b|soul|blues') | new_viral_all_df['Genre 3'].str.contains('r&b|soul|blues')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('pop') | new_viral_all_df['Genre 2'].str.contains('pop') | new_viral_all_df['Genre 3'].str.contains('pop')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('rock') | new_viral_all_df['Genre 2'].str.contains('rock') | new_viral_all_df['Genre 3'].str.contains('rock')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('country') | new_viral_all_df['Genre 2'].str.contains('country') | new_viral_all_df['Genre 3'].str.contains('country')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('latin') | new_viral_all_df['Genre 2'].str.contains('latin') | new_viral_all_df['Genre 3'].str.contains('latin')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('disco') | new_viral_all_df['Genre 2'].str.contains('disco') | new_viral_all_df['Genre 3'].str.contains('disco')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('indie') | new_viral_all_df['Genre 2'].str.contains('indie') | new_viral_all_df['Genre 3'].str.contains('indie')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('house|edm') | new_viral_all_df['Genre 2'].str.contains('house|edm') | new_viral_all_df['Genre 3'].str.contains('house|edm')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('alt|alternative') | new_viral_all_df['Genre 2'].str.contains('alt|alternative') | new_viral_all_df['Genre 3'].str.contains('alt|alternative')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('glitchcore') | new_viral_all_df['Genre 2'].str.contains('glitchcore') | new_viral_all_df['Genre 3'].str.contains('glitchcore')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('brooklyn drill') | new_viral_all_df['Genre 2'].str.contains('brooklyn drill') | new_viral_all_df['Genre 3'].str.contains('brooklyn drill')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('halloween') | new_viral_all_df['Genre 2'].str.contains('halloween') | new_viral_all_df['Genre 3'].str.contains('halloween')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('christmas') | new_viral_all_df['Genre 2'].str.contains('christmas') | new_viral_all_df['Genre 3'].str.contains('christmas')]), len(new_viral_all_df[new_viral_all_df['Genre 1'].str.contains('classical') | new_viral_all_df['Genre 2'].str.contains('classical') | new_viral_all_df['Genre 3'].str.contains('classical')])]
}
all_viral_genre_df = pd.DataFrame(genre_data)
all_viral_genre_df.head()

In [ ]:
max_count = all_viral_genre_df['Count'].max()
print(max_count)
percent_total_max = (max_count/all_viral_genre_df['Count'].sum())*100
print(percent_total_max)

In [ ]:
all_viral_genre_df['Percent']=(all_viral_genre_df['Count']/all_viral_genre_df['Count'].sum())*100

In [ ]:
all_viral_genre_df

In [ ]:
sorted_all_viral_genre_df = all_viral_genre_df.sort_values('Count', ascending=False)

### <u>Top Viral Genres Distribution Throughout 2020 (Sampled Months)</u>

In [ ]:
Assign specific colors to genres

In [ ]:
genres = ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical']
colors = ['red', 'blue', 'purple', 'orchid', 'orange', 'indianred', 'gold', 'seagreen', 'slateblue', 'coral', 'silver', 'palevioletred', 'limegreen', 'maroon', 'purple']

In [ ]:
genres_all = sorted_all_viral_genre_df['Genre'].tolist()
all_colors = []

for x in genres_all:
    if x == genres[0]:
        all_colors.append(colors[0])
    if x == genres[1]:
        all_colors.append(colors[1])
    if x == genres[2]:
        all_colors.append(colors[2])      
    if x == genres[3]:
        all_colors.append(colors[3]) 
    if x == genres[4]:
        all_colors.append(colors[4])
    if x == genres[5]:
        all_colors.append(colors[5])
    if x == genres[6]:
        all_colors.append(colors[6])
    if x == genres[7]:
        all_colors.append(colors[7]) 
    if x == genres[8]:
        all_colors.append(colors[8])
    if x == genres[9]:
        all_colors.append(colors[9])
    if x == genres[10]:
        all_colors.append(colors[10])
    if x == genres[11]:
        all_colors.append(colors[11])
    if x == genres[12]:
        all_colors.append(colors[12])
    if x == genres[13]:
        all_colors.append(colors[13])
    if x == genres[14]:
        all_colors.append(colors[14])

In [ ]:
names = sorted_all_viral_genre_df['Genre']
values = sorted_all_viral_genre_df['Count'].astype(float)

plt.bar(names, values, color=all_colors, zorder=2)
plt.xticks(rotation=90, size=14)
plt.grid(alpha=0.5, zorder=0)
plt.yticks(size=14)
plt.ylabel("Counts", size=14)
plt.title("Total Distribution of Viral Song Genres for Sampled Months in 2020", size=18)

plt.gcf().set_size_inches(15,10)
plt.savefig('total_all_viral_genre.jpg', facecolor='w')
plt.show()

### Distribution of Genres per Month

Create and clean dataframes. Plethora of odd genres unified into more general categories (original genre name from API reques was used as guide)

In [ ]:
jan_df = new_viral_all_df[new_viral_all_df['Month'] == 'January']
jan_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(jan_df[jan_df['Genre 1'].str.contains('rap|hip hop|trap') | jan_df['Genre 2'].str.contains('rap|hip hop|trap') | jan_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(jan_df[jan_df['Genre 1'].str.contains('r&b|soul|blues') | jan_df['Genre 2'].str.contains('r&b|soul|blues') | jan_df['Genre 3'].str.contains('r&b|soul|blues')]), len(jan_df[jan_df['Genre 1'].str.contains('pop') | jan_df['Genre 2'].str.contains('pop') | jan_df['Genre 3'].str.contains('pop')]), len(jan_df[jan_df['Genre 1'].str.contains('rock') | jan_df['Genre 2'].str.contains('rock') | jan_df['Genre 3'].str.contains('rock')]), len(jan_df[jan_df['Genre 1'].str.contains('country') | jan_df['Genre 2'].str.contains('country') | jan_df['Genre 3'].str.contains('country')]), len(jan_df[jan_df['Genre 1'].str.contains('latin') | jan_df['Genre 2'].str.contains('latin') | jan_df['Genre 3'].str.contains('latin')]), len(jan_df[jan_df['Genre 1'].str.contains('disco') | jan_df['Genre 2'].str.contains('disco') | jan_df['Genre 3'].str.contains('disco')]), len(jan_df[jan_df['Genre 1'].str.contains('indie') | jan_df['Genre 2'].str.contains('indie') | jan_df['Genre 3'].str.contains('indie')]), len(jan_df[jan_df['Genre 1'].str.contains('house|edm') | jan_df['Genre 2'].str.contains('house|edm') | jan_df['Genre 3'].str.contains('house|edm')]), len(jan_df[jan_df['Genre 1'].str.contains('alt|alternative') | jan_df['Genre 2'].str.contains('alt|alternative') | jan_df['Genre 3'].str.contains('alt|alternative')]), len(jan_df[jan_df['Genre 1'].str.contains('glitchcore') | jan_df['Genre 2'].str.contains('glitchcore') | jan_df['Genre 3'].str.contains('glitchcore')]), len(jan_df[jan_df['Genre 1'].str.contains('brooklyn drill') | jan_df['Genre 2'].str.contains('brooklyn drill') | jan_df['Genre 3'].str.contains('brooklyn drill')]), len(jan_df[jan_df['Genre 1'].str.contains('halloween') | jan_df['Genre 2'].str.contains('halloween') | jan_df['Genre 3'].str.contains('halloween')]), len(jan_df[jan_df['Genre 1'].str.contains('christmas') | jan_df['Genre 2'].str.contains('christmas') | jan_df['Genre 3'].str.contains('christmas')]), len(jan_df[jan_df['Genre 1'].str.contains('classical') | jan_df['Genre 2'].str.contains('classical') | jan_df['Genre 3'].str.contains('classical')])]
}
jan_viral_genre_df = pd.DataFrame(genre_data)
jan_viral_genre_df.head()

In [ ]:
sorted_jan_viral_genre_df = jan_viral_genre_df.sort_values('Count', ascending=False)
sorted_jan_viral_genre_df.head()

In [ ]:
mar_df = new_viral_all_df[new_viral_all_df['Month'] == 'March']
mar_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(mar_df[mar_df['Genre 1'].str.contains('rap|hip hop|trap') | mar_df['Genre 2'].str.contains('rap|hip hop|trap') | mar_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(mar_df[mar_df['Genre 1'].str.contains('r&b|soul|blues') | mar_df['Genre 2'].str.contains('r&b|soul|blues') | mar_df['Genre 3'].str.contains('r&b|soul|blues')]), len(mar_df[mar_df['Genre 1'].str.contains('pop') | mar_df['Genre 2'].str.contains('pop') | mar_df['Genre 3'].str.contains('pop')]), len(mar_df[mar_df['Genre 1'].str.contains('rock') | mar_df['Genre 2'].str.contains('rock') | mar_df['Genre 3'].str.contains('rock')]), len(mar_df[mar_df['Genre 1'].str.contains('country') | mar_df['Genre 2'].str.contains('country') | mar_df['Genre 3'].str.contains('country')]), len(mar_df[mar_df['Genre 1'].str.contains('latin') | mar_df['Genre 2'].str.contains('latin') | mar_df['Genre 3'].str.contains('latin')]), len(mar_df[mar_df['Genre 1'].str.contains('disco') | mar_df['Genre 2'].str.contains('disco') | mar_df['Genre 3'].str.contains('disco')]), len(mar_df[mar_df['Genre 1'].str.contains('indie') | mar_df['Genre 2'].str.contains('indie') | mar_df['Genre 3'].str.contains('indie')]), len(mar_df[mar_df['Genre 1'].str.contains('house|edm') | mar_df['Genre 2'].str.contains('house|edm') | mar_df['Genre 3'].str.contains('house|edm')]), len(mar_df[mar_df['Genre 1'].str.contains('alt|alternative') | mar_df['Genre 2'].str.contains('alt|alternative') | mar_df['Genre 3'].str.contains('alt|alternative')]), len(mar_df[mar_df['Genre 1'].str.contains('glitchcore') | mar_df['Genre 2'].str.contains('glitchcore') | mar_df['Genre 3'].str.contains('glitchcore')]), len(mar_df[mar_df['Genre 1'].str.contains('brooklyn drill') | mar_df['Genre 2'].str.contains('brooklyn drill') | mar_df['Genre 3'].str.contains('brooklyn drill')]), len(mar_df[mar_df['Genre 1'].str.contains('halloween') | mar_df['Genre 2'].str.contains('halloween') | mar_df['Genre 3'].str.contains('halloween')]), len(mar_df[mar_df['Genre 1'].str.contains('christmas') | mar_df['Genre 2'].str.contains('christmas') | mar_df['Genre 3'].str.contains('christmas')]), len(mar_df[mar_df['Genre 1'].str.contains('classical') | mar_df['Genre 2'].str.contains('classical') | mar_df['Genre 3'].str.contains('classical')])]
}
mar_viral_genre_df = pd.DataFrame(genre_data)
mar_viral_genre_df.head()

In [ ]:
sorted_mar_viral_genre_df = mar_viral_genre_df.sort_values('Count', ascending=False)
sorted_mar_viral_genre_df.head()

In [ ]:
may_df = new_viral_all_df[new_viral_all_df['Month'] == 'May']
may_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(may_df[may_df['Genre 1'].str.contains('rap|hip hop|trap') | may_df['Genre 2'].str.contains('rap|hip hop|trap') | may_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(may_df[may_df['Genre 1'].str.contains('r&b|soul|blues') | may_df['Genre 2'].str.contains('r&b|soul|blues') | may_df['Genre 3'].str.contains('r&b|soul|blues')]), len(may_df[may_df['Genre 1'].str.contains('pop') | may_df['Genre 2'].str.contains('pop') | may_df['Genre 3'].str.contains('pop')]), len(may_df[may_df['Genre 1'].str.contains('rock') | may_df['Genre 2'].str.contains('rock') | may_df['Genre 3'].str.contains('rock')]), len(may_df[may_df['Genre 1'].str.contains('country') | may_df['Genre 2'].str.contains('country') | may_df['Genre 3'].str.contains('country')]), len(may_df[may_df['Genre 1'].str.contains('latin') | may_df['Genre 2'].str.contains('latin') | may_df['Genre 3'].str.contains('latin')]), len(may_df[may_df['Genre 1'].str.contains('disco') | may_df['Genre 2'].str.contains('disco') | may_df['Genre 3'].str.contains('disco')]), len(may_df[may_df['Genre 1'].str.contains('indie') | may_df['Genre 2'].str.contains('indie') | may_df['Genre 3'].str.contains('indie')]), len(may_df[may_df['Genre 1'].str.contains('house|edm') | may_df['Genre 2'].str.contains('house|edm') | may_df['Genre 3'].str.contains('house|edm')]), len(may_df[may_df['Genre 1'].str.contains('alt|alternative') | may_df['Genre 2'].str.contains('alt|alternative') | may_df['Genre 3'].str.contains('alt|alternative')]), len(may_df[may_df['Genre 1'].str.contains('glitchcore') | may_df['Genre 2'].str.contains('glitchcore') | may_df['Genre 3'].str.contains('glitchcore')]), len(may_df[may_df['Genre 1'].str.contains('brooklyn drill') | may_df['Genre 2'].str.contains('brooklyn drill') | may_df['Genre 3'].str.contains('brooklyn drill')]), len(may_df[may_df['Genre 1'].str.contains('halloween') | may_df['Genre 2'].str.contains('halloween') | may_df['Genre 3'].str.contains('halloween')]), len(may_df[may_df['Genre 1'].str.contains('christmas') | may_df['Genre 2'].str.contains('christmas') | may_df['Genre 3'].str.contains('christmas')]), len(may_df[may_df['Genre 1'].str.contains('classical') | may_df['Genre 2'].str.contains('classical') | may_df['Genre 3'].str.contains('classical')])]
}
may_viral_genre_df = pd.DataFrame(genre_data)
may_viral_genre_df.head()

In [ ]:
sorted_may_viral_genre_df = may_viral_genre_df.sort_values('Count', ascending=False)
sorted_may_viral_genre_df.head()

In [ ]:
jul_df = new_viral_all_df[new_viral_all_df['Month'] == 'July']
jul_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(jul_df[jul_df['Genre 1'].str.contains('rap|hip hop|trap') | jul_df['Genre 2'].str.contains('rap|hip hop|trap') | jul_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(jul_df[jul_df['Genre 1'].str.contains('r&b|soul|blues') | jul_df['Genre 2'].str.contains('r&b|soul|blues') | jul_df['Genre 3'].str.contains('r&b|soul|blues')]), len(jul_df[jul_df['Genre 1'].str.contains('pop') | jul_df['Genre 2'].str.contains('pop') | jul_df['Genre 3'].str.contains('pop')]), len(jul_df[jul_df['Genre 1'].str.contains('rock') | jul_df['Genre 2'].str.contains('rock') | jul_df['Genre 3'].str.contains('rock')]), len(jul_df[jul_df['Genre 1'].str.contains('country') | jul_df['Genre 2'].str.contains('country') | jul_df['Genre 3'].str.contains('country')]), len(jul_df[jul_df['Genre 1'].str.contains('latin') | jul_df['Genre 2'].str.contains('latin') | jul_df['Genre 3'].str.contains('latin')]), len(jul_df[jul_df['Genre 1'].str.contains('disco') | jul_df['Genre 2'].str.contains('disco') | jul_df['Genre 3'].str.contains('disco')]), len(jul_df[jul_df['Genre 1'].str.contains('indie') | jul_df['Genre 2'].str.contains('indie') | jul_df['Genre 3'].str.contains('indie')]), len(jul_df[jul_df['Genre 1'].str.contains('house|edm') | jul_df['Genre 2'].str.contains('house|edm') | jul_df['Genre 3'].str.contains('house|edm')]), len(jul_df[jul_df['Genre 1'].str.contains('alt|alternative') | jul_df['Genre 2'].str.contains('alt|alternative') | jul_df['Genre 3'].str.contains('alt|alternative')]), len(jul_df[jul_df['Genre 1'].str.contains('glitchcore') | jul_df['Genre 2'].str.contains('glitchcore') | jul_df['Genre 3'].str.contains('glitchcore')]), len(jul_df[jul_df['Genre 1'].str.contains('brooklyn drill') | jul_df['Genre 2'].str.contains('brooklyn drill') | jul_df['Genre 3'].str.contains('brooklyn drill')]), len(jul_df[jul_df['Genre 1'].str.contains('halloween') | jul_df['Genre 2'].str.contains('halloween') | jul_df['Genre 3'].str.contains('halloween')]), len(jul_df[jul_df['Genre 1'].str.contains('christmas') | jul_df['Genre 2'].str.contains('christmas') | jul_df['Genre 3'].str.contains('christmas')]), len(jul_df[jul_df['Genre 1'].str.contains('classical') | jul_df['Genre 2'].str.contains('classical') | jul_df['Genre 3'].str.contains('classical')])]
}
jul_viral_genre_df = pd.DataFrame(genre_data)
jul_viral_genre_df.head()

In [ ]:
sorted_jul_viral_genre_df = jul_viral_genre_df.sort_values('Count', ascending=False)
sorted_jul_viral_genre_df.head()

In [ ]:
sep_df = new_viral_all_df[new_viral_all_df['Month'] == 'September']
sep_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(sep_df[sep_df['Genre 1'].str.contains('rap|hip hop|trap') | sep_df['Genre 2'].str.contains('rap|hip hop|trap') | sep_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(sep_df[sep_df['Genre 1'].str.contains('r&b|soul|blues') | sep_df['Genre 2'].str.contains('r&b|soul|blues') | sep_df['Genre 3'].str.contains('r&b|soul|blues')]), len(sep_df[sep_df['Genre 1'].str.contains('pop') | sep_df['Genre 2'].str.contains('pop') | sep_df['Genre 3'].str.contains('pop')]), len(sep_df[sep_df['Genre 1'].str.contains('rock') | sep_df['Genre 2'].str.contains('rock') | sep_df['Genre 3'].str.contains('rock')]), len(sep_df[sep_df['Genre 1'].str.contains('country') | sep_df['Genre 2'].str.contains('country') | sep_df['Genre 3'].str.contains('country')]), len(sep_df[sep_df['Genre 1'].str.contains('latin') | sep_df['Genre 2'].str.contains('latin') | sep_df['Genre 3'].str.contains('latin')]), len(sep_df[sep_df['Genre 1'].str.contains('disco') | sep_df['Genre 2'].str.contains('disco') | sep_df['Genre 3'].str.contains('disco')]), len(sep_df[sep_df['Genre 1'].str.contains('indie') | sep_df['Genre 2'].str.contains('indie') | sep_df['Genre 3'].str.contains('indie')]), len(sep_df[sep_df['Genre 1'].str.contains('house|edm') | sep_df['Genre 2'].str.contains('house|edm') | sep_df['Genre 3'].str.contains('house|edm')]), len(sep_df[sep_df['Genre 1'].str.contains('alt|alternative') | sep_df['Genre 2'].str.contains('alt|alternative') | sep_df['Genre 3'].str.contains('alt|alternative')]), len(sep_df[sep_df['Genre 1'].str.contains('glitchcore') | sep_df['Genre 2'].str.contains('glitchcore') | sep_df['Genre 3'].str.contains('glitchcore')]), len(sep_df[sep_df['Genre 1'].str.contains('brooklyn drill') | sep_df['Genre 2'].str.contains('brooklyn drill') | sep_df['Genre 3'].str.contains('brooklyn drill')]), len(sep_df[sep_df['Genre 1'].str.contains('halloween') | sep_df['Genre 2'].str.contains('halloween') | sep_df['Genre 3'].str.contains('halloween')]), len(sep_df[sep_df['Genre 1'].str.contains('christmas') | sep_df['Genre 2'].str.contains('christmas') | sep_df['Genre 3'].str.contains('christmas')]), len(sep_df[sep_df['Genre 1'].str.contains('classical') | sep_df['Genre 2'].str.contains('classical') | sep_df['Genre 3'].str.contains('classical')])]
}
sep_viral_genre_df = pd.DataFrame(genre_data)
sep_viral_genre_df.head()

In [ ]:
sorted_sep_viral_genre_df = sep_viral_genre_df.sort_values('Count', ascending=False)
sorted_sep_viral_genre_df.head()

In [ ]:
nov_df = new_viral_all_df[new_viral_all_df['Month'] == 'November']
nov_df.head()

In [ ]:
genre_data = {
    'Genre': ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical'],
    'Count': [len(nov_df[nov_df['Genre 1'].str.contains('rap|hip hop|trap') | nov_df['Genre 2'].str.contains('rap|hip hop|trap') | nov_df['Genre 3'].str.contains('rap|hip hop|trap')]), len(nov_df[nov_df['Genre 1'].str.contains('r&b|soul|blues') | nov_df['Genre 2'].str.contains('r&b|soul|blues') | nov_df['Genre 3'].str.contains('r&b|soul|blues')]), len(nov_df[nov_df['Genre 1'].str.contains('pop') | nov_df['Genre 2'].str.contains('pop') | nov_df['Genre 3'].str.contains('pop')]), len(nov_df[nov_df['Genre 1'].str.contains('rock') | nov_df['Genre 2'].str.contains('rock') | nov_df['Genre 3'].str.contains('rock')]), len(nov_df[nov_df['Genre 1'].str.contains('country') | nov_df['Genre 2'].str.contains('country') | nov_df['Genre 3'].str.contains('country')]), len(nov_df[nov_df['Genre 1'].str.contains('latin') | nov_df['Genre 2'].str.contains('latin') | nov_df['Genre 3'].str.contains('latin')]), len(nov_df[nov_df['Genre 1'].str.contains('disco') | nov_df['Genre 2'].str.contains('disco') | nov_df['Genre 3'].str.contains('disco')]), len(nov_df[nov_df['Genre 1'].str.contains('indie') | nov_df['Genre 2'].str.contains('indie') | nov_df['Genre 3'].str.contains('indie')]), len(nov_df[nov_df['Genre 1'].str.contains('house|edm') | nov_df['Genre 2'].str.contains('house|edm') | nov_df['Genre 3'].str.contains('house|edm')]), len(nov_df[nov_df['Genre 1'].str.contains('alt|alternative') | nov_df['Genre 2'].str.contains('alt|alternative') | nov_df['Genre 3'].str.contains('alt|alternative')]), len(nov_df[nov_df['Genre 1'].str.contains('glitchcore') | nov_df['Genre 2'].str.contains('glitchcore') | nov_df['Genre 3'].str.contains('glitchcore')]), len(nov_df[nov_df['Genre 1'].str.contains('brooklyn drill') | nov_df['Genre 2'].str.contains('brooklyn drill') | nov_df['Genre 3'].str.contains('brooklyn drill')]), len(nov_df[nov_df['Genre 1'].str.contains('halloween') | nov_df['Genre 2'].str.contains('halloween') | nov_df['Genre 3'].str.contains('halloween')]), len(nov_df[nov_df['Genre 1'].str.contains('christmas') | nov_df['Genre 2'].str.contains('christmas') | nov_df['Genre 3'].str.contains('christmas')]), len(nov_df[nov_df['Genre 1'].str.contains('classical') | nov_df['Genre 2'].str.contains('classical') | nov_df['Genre 3'].str.contains('classical')])]
}
nov_viral_genre_df = pd.DataFrame(genre_data)
nov_viral_genre_df.head()

In [ ]:
sorted_nov_viral_genre_df = nov_viral_genre_df.sort_values('Count', ascending=False)
sorted_nov_viral_genre_df.head()

Color coding all the genres

In [ ]:
sorted_jan_viral_genre_df["Colors"] = ""
sorted_jan_viral_genre_df

In [ ]:
genres = ['Rap/Hip Hop/Trap', 'R&B/Soul/Blues',  'Pop', 'Rock', 'Country', 'Latin', 'Disco', 'Indie', 'House/EDM', 'Alternative/Alt', 'Glitchcore', 'Brooklyn Drill', 'Halloween', 'Christmas', 'Classical']
colors = ['red', 'blue', 'purple', 'orchid', 'orange', 'indianred', 'gold', 'seagreen', 'slateblue', 'coral', 'silver', 'palevioletred', 'limegreen', 'maroon', 'purple']

In [ ]:
genres_jan = sorted_jan_viral_genre_df['Genre'].tolist()
jan_colors = []

for x in genres_jan:
    if x == genres[0]:
        jan_colors.append(colors[0])
    if x == genres[1]:
        jan_colors.append(colors[1])
    if x == genres[2]:
        jan_colors.append(colors[2])      
    if x == genres[3]:
        jan_colors.append(colors[3]) 
    if x == genres[4]:
        jan_colors.append(colors[4])
    if x == genres[5]:
        jan_colors.append(colors[5])
    if x == genres[6]:
        jan_colors.append(colors[6])
    if x == genres[7]:
        jan_colors.append(colors[7]) 
    if x == genres[8]:
        jan_colors.append(colors[8])
    if x == genres[9]:
        jan_colors.append(colors[9])
    if x == genres[10]:
        jan_colors.append(colors[10])
    if x == genres[11]:
        jan_colors.append(colors[11])
    if x == genres[12]:
        jan_colors.append(colors[12])
    if x == genres[13]:
        jan_colors.append(colors[13])
    if x == genres[14]:
        jan_colors.append(colors[14])

In [ ]:
genres_mar = sorted_mar_viral_genre_df['Genre'].tolist()
mar_colors = []

for x in genres_mar:
    if x == genres[0]:
        mar_colors.append(colors[0])
    if x == genres[1]:
        mar_colors.append(colors[1])
    if x == genres[2]:
        mar_colors.append(colors[2])      
    if x == genres[3]:
        mar_colors.append(colors[3]) 
    if x == genres[4]:
        mar_colors.append(colors[4])
    if x == genres[5]:
        mar_colors.append(colors[5])
    if x == genres[6]:
        mar_colors.append(colors[6])
    if x == genres[7]:
        mar_colors.append(colors[7]) 
    if x == genres[8]:
        mar_colors.append(colors[8])
    if x == genres[9]:
        mar_colors.append(colors[9])
    if x == genres[10]:
        mar_colors.append(colors[10])
    if x == genres[11]:
        mar_colors.append(colors[11])
    if x == genres[12]:
        mar_colors.append(colors[12])
    if x == genres[13]:
        mar_colors.append(colors[13])
    if x == genres[14]:
        mar_colors.append(colors[14])

In [ ]:
genres_may = sorted_may_viral_genre_df['Genre'].tolist()
may_colors = []

for x in genres_may:
    if x == genres[0]:
        may_colors.append(colors[0])
    if x == genres[1]:
        may_colors.append(colors[1])
    if x == genres[2]:
        may_colors.append(colors[2])      
    if x == genres[3]:
        may_colors.append(colors[3]) 
    if x == genres[4]:
        may_colors.append(colors[4])
    if x == genres[5]:
        may_colors.append(colors[5])
    if x == genres[6]:
        may_colors.append(colors[6])
    if x == genres[7]:
        may_colors.append(colors[7]) 
    if x == genres[8]:
        may_colors.append(colors[8])
    if x == genres[9]:
        may_colors.append(colors[9])
    if x == genres[10]:
        may_colors.append(colors[10])
    if x == genres[11]:
        may_colors.append(colors[11])
    if x == genres[12]:
        may_colors.append(colors[12])
    if x == genres[13]:
        may_colors.append(colors[13])
    if x == genres[14]:
        may_colors.append(colors[14])

In [ ]:
genres_jul = sorted_jul_viral_genre_df['Genre'].tolist()
jul_colors = []

for x in genres_jul:
    if x == genres[0]:
        jul_colors.append(colors[0])
    if x == genres[1]:
        jul_colors.append(colors[1])
    if x == genres[2]:
        jul_colors.append(colors[2])      
    if x == genres[3]:
        jul_colors.append(colors[3]) 
    if x == genres[4]:
        jul_colors.append(colors[4])
    if x == genres[5]:
        jul_colors.append(colors[5])
    if x == genres[6]:
        jul_colors.append(colors[6])
    if x == genres[7]:
        jul_colors.append(colors[7]) 
    if x == genres[8]:
        jul_colors.append(colors[8])
    if x == genres[9]:
        jul_colors.append(colors[9])
    if x == genres[10]:
        jul_colors.append(colors[10])
    if x == genres[11]:
        jul_colors.append(colors[11])
    if x == genres[12]:
        jul_colors.append(colors[12])
    if x == genres[13]:
        jul_colors.append(colors[13])
    if x == genres[14]:
        jul_colors.append(colors[14])

In [ ]:
genres_sep = sorted_sep_viral_genre_df['Genre'].tolist()
sep_colors = []

for x in genres_sep:
    if x == genres[0]:
        sep_colors.append(colors[0])
    if x == genres[1]:
        sep_colors.append(colors[1])
    if x == genres[2]:
        sep_colors.append(colors[2])      
    if x == genres[3]:
        sep_colors.append(colors[3]) 
    if x == genres[4]:
        sep_colors.append(colors[4])
    if x == genres[5]:
        sep_colors.append(colors[5])
    if x == genres[6]:
        sep_colors.append(colors[6])
    if x == genres[7]:
        sep_colors.append(colors[7]) 
    if x == genres[8]:
        sep_colors.append(colors[8])
    if x == genres[9]:
        sep_colors.append(colors[9])
    if x == genres[10]:
        sep_colors.append(colors[10])
    if x == genres[11]:
        sep_colors.append(colors[11])
    if x == genres[12]:
        sep_colors.append(colors[12])
    if x == genres[13]:
        sep_colors.append(colors[13])
    if x == genres[14]:
        sep_colors.append(colors[14])

In [ ]:
genres_nov = sorted_nov_viral_genre_df['Genre'].tolist()
nov_colors = []

for x in genres_nov:
    if x == genres[0]:
        nov_colors.append(colors[0])
    if x == genres[1]:
        nov_colors.append(colors[1])
    if x == genres[2]:
        nov_colors.append(colors[2])      
    if x == genres[3]:
        nov_colors.append(colors[3]) 
    if x == genres[4]:
        nov_colors.append(colors[4])
    if x == genres[5]:
        nov_colors.append(colors[5])
    if x == genres[6]:
        nov_colors.append(colors[6])
    if x == genres[7]:
        nov_colors.append(colors[7]) 
    if x == genres[8]:
        nov_colors.append(colors[8])
    if x == genres[9]:
        nov_colors.append(colors[9])
    if x == genres[10]:
        nov_colors.append(colors[10])
    if x == genres[11]:
        nov_colors.append(colors[11])
    if x == genres[12]:
        nov_colors.append(colors[12])
    if x == genres[13]:
        nov_colors.append(colors[13])
    if x == genres[14]:
        nov_colors.append(colors[14])

Create bar charts as subplots for visualization of genre behavior across months.

In [ ]:
jan_x = sorted_jan_viral_genre_df['Genre']
jan_y = sorted_jan_viral_genre_df['Count']

mar_x = sorted_mar_viral_genre_df['Genre']
mar_y = sorted_mar_viral_genre_df['Count']

may_x = sorted_may_viral_genre_df['Genre']
may_y = sorted_may_viral_genre_df['Count']

jul_x = sorted_jul_viral_genre_df['Genre']
jul_y = sorted_jul_viral_genre_df['Count']

sep_x = sorted_sep_viral_genre_df['Genre']
sep_y = sorted_sep_viral_genre_df['Count']

nov_x = sorted_nov_viral_genre_df['Genre']
nov_y = sorted_nov_viral_genre_df['Count']

# fig, (ax1,ax2,ax3,ax4,ax5,ax6) = plt.subplots(2,3, sharey=True)
fig = plt.figure(figsize = (20, 15))
fig.add_axes()

ax1 = fig.add_subplot(231)
ax1.bar(jan_x, jan_y, color=jan_colors, zorder=2)
ax1.grid(alpha=0.5, zorder=0)
ax1.xaxis.set(ticks=range(len(jan_x)))
# ax1.setxticklabels(jan_x, rotation=90)
ax1.tick_params(axis='y', direction='inout', length=10)
ax1.tick_params(axis='x', direction='inout', length=10, labelrotation=90)
ax1.set(title="January")
# ax1.set_xlabel('Genre', fontsize=14)
ax1.set_ylabel('Counts', fontsize=14)

ax2 = fig.add_subplot(232, sharey=ax1)
ax2.bar(mar_x, mar_y, color=mar_colors, zorder=2)
ax2.grid(alpha=0.5, zorder=0)
ax2.xaxis.set(ticks=range(len(mar_x)))
# ax1.setxticklabels(jan_x, rotation=90)
# ax2.tick_params(axis='y', direction='inout', length=10)
ax2.tick_params(axis='x', direction='inout', length=10, labelrotation=90)
ax2.set(title="March")
# ax2.set_xlabel('Genre', fontsize=14)
# ax2.set_ylabel('Counts', fontsize=14)


ax3 = fig.add_subplot(233, sharey=ax1)
ax3.bar(may_x, may_y, color=may_colors, zorder=2)
ax3.grid(alpha=0.5, zorder=0)
ax3.xaxis.set(ticks=range(len(may_x)))
# ax1.setxticklabels(jan_x, rotation=90)
# ax3.tick_params(axis='y', direction='inout', length=10)
ax3.tick_params(axis='x', direction='inout', length=10, labelrotation=90)
ax3.set(title="May")
# ax3.set_xlabel('Genre', fontsize=14)
# ax3.set_ylabel('Counts', fontsize=14)


ax4 = fig.add_subplot(234)
ax4.bar(jul_x, jul_y, color=jul_colors, zorder=2)
ax4.grid(alpha=0.5, zorder=0)
ax4.xaxis.set(ticks=range(len(jul_x)))
# ax1.setxticklabels(jan_x, rotation=90)
ax4.tick_params(axis='y', direction='inout', length=10)
ax4.tick_params(axis='x', direction='inout', length=10, labelrotation=90)
ax4.set(title="July")
# ax4.set_xlabel('Genre', fontsize=14)
ax4.set_ylabel('Counts', fontsize=14)


ax5 = fig.add_subplot(235, sharey=ax4)
ax5.bar(sep_x, sep_y, color=sep_colors, zorder=2)
ax5.grid(alpha=0.5, zorder=0)
ax5.xaxis.set(ticks=range(len(sep_x)))
# ax1.setxticklabels(jan_x, rotation=90)
# ax5.tick_params(axis='y', direction='inout', length=10)
ax5.tick_params(axis='x', direction='inout', length=10, labelrotation=90)
ax5.set(title="September")
# ax5.set_xlabel('Genre', fontsize=14)
# ax5.set_ylabel('Counts', fontsize=14)


ax6 = fig.add_subplot(236, sharey=ax4)
ax6.bar(nov_x, nov_y, color=nov_colors, zorder=2)
ax6.grid(alpha=0.5, zorder=0)
ax6.xaxis.set(ticks=range(len(nov_x)))
# ax1.setxticklabels(jan_x, rotation=90)
# ax6.tick_params(axis='y', direction='inout', length=10)
ax6.tick_params(axis='x', direction='inout', length=10, labelrotation=90)
ax6.set(title="November")
# ax6.set_xlabel('Genre', fontsize=12)
# ax6.set_ylabel('Counts', fontsize=14)

fig.suptitle('Distribution of Viral Song Genres in 2020', fontsize=14)

plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.15, hspace=0.5)
# plt.bar(names, values, color=['red', 'blue', 'purple', 'orchid', 'orange', 'indianred', 'gold', 'seagreen', 'slateblue', 'coral', 'silver', 'palevioletred', 'limegreen'], zorder=2)
# plt.xticks(rotation=90, size=14)
# plt.grid(alpha=0.5, zorder=0)
# plt.yticks(size=14)
# plt.ylabel("Counts", size=14)
# plt.title("Total Distribution of Viral Song Genres for Sampled Months in 2020", size=18)
# plt.setp(axa.xaxis.get_majorticklabels(), rotation=95)
# plt.subplot(221)

plt.gcf()
plt.show()
fig.savefig('total_months_viral_genre.jpg', facecolor='w', dpi=400)

## Unemployment Data

In [ ]:
## Getting CSV, creating dataframes, cleaning and plotting the data

In [ ]:
ui_df = pd.read_csv('csv file/UI_weekly_2020.csv', skiprows=[0,1],skip_blank_lines=True, skipinitialspace=True)
ui_df.head()

In [ ]:
ui_df = ui_df.rename(columns={'Unnamed: 0': "Date"})
ui_df.head()

In [ ]:
ui_df.convert_dtypes(infer_objects=True, convert_integer=True)

In [ ]:
ui_df_dropped = ui_df.dropna(how='all')
ui_df_dropped.apply(pd.to_numeric, errors='coerce')
ui_df_dropped

In [ ]:
ui2_df_dropped = ui_df_dropped[(ui_df_dropped.Date != 'Run Date: 4/28/2021')]
ui2_df_dropped

In [ ]:
new_ui_df = ui2_df_dropped.copy()
new_ui_df['Week Number'] = ""
new_ui_df['Date'] = pd.to_datetime(new_ui_df['Date'], errors='coerce')
new_ui_df['N.S.A'] = new_ui_df['N.S.A'].str.replace(',', '').astype(int)
new_ui_df['S.A.'] = new_ui_df['S.A.'].str.replace(',', '').astype(int) # to completely convert it to a string
new_ui_df["Week Number"] = new_ui_df['Date'].dt.week
new_ui_df

In [ ]:
week_num = new_ui_df["Week Number"].tolist()
print(week_num)

Obtaining top 200 information to obtain the strea numbers per week to put the graph comparing its trend to unemployment claims

In [ ]:
streams_all = pd.read_csv('csv files/top200_all_months.csv')
streams_all = streams_all.drop(["Unnamed: 0"], axis=1)

streams_all.head()

In [ ]:
streams_all_sum = streams_all.groupby('Week Number')['Streams'].sum()
print(streams_all_sum)

In [ ]:
num_streams = streams_all_sum.astype(float).to_list()
weeks = pd.DataFrame(streams_all.groupby('Week Number').count())
print(num_streams)
weeks.head()

In [ ]:
weeks_list = weeks_reindex['Week Number'].to_list()
print(weeks_list)
print(len(num_streams))
print(len(weeks_list))

Create unemployment claims plot

In [ ]:
x_UI= week_num
y_UI= new_ui_df['S.A.']

fig = plt.figure(figsize = (15, 20))
fig.add_axes()

ax1 = fig.add_subplot(211)
ax1.plot(x_UI, y_UI, zorder=2, color='red')
ax1.grid(alpha=0.5, zorder=0)
ax1.xaxis.set(ticks=range(len(x_UI)))


ax1.set_xlabel('Weeks', fontsize=16)

ax1.tick_params(axis='y', direction='inout', length=10, size=14)
ax1.tick_params(axis='x', direction='inout', length=10, size=14)
# ax1.set(title="Weekly Unemployment Insurance Claims in 2020")
ax1.set_title('Weekly Unemployment Insurance Claims in 2020', fontsize=20)

ax1.set_ylabel('Number of Weekly Unemployment Claims (Seasonally Adjusted)', fontsize=16)

# plt.gcf().set_size_inches(20,20)
plt.tight_layout()
fig.savefig('2020_Weekly_UI.png', facecolor='w', dpi=600)
plt.show()

Combine bar graph of streams per week and line graph of unemplyment claims

In [ ]:

#https://www.geeksforgeeks.org/how-to-add-a-y-axis-label-to-the-secondary-y-axis-in-matplotlib/
x = week_num
x2 = weeks_list
# y-axis values
y1 = new_ui_df['S.A.']
  
# secondary y-axis values
y2 = num_streams
  
# plotting figures by creating aexs object
# using subplots() function
fig, ax = plt.subplots(figsize = (10, 5))
plt.title('Unemployment Claims and Streams per Week', size=20)
  
# using the twinx() for creating another
# axes object for secondry y-Axis
ax2 = ax.twinx()
ax.plot(x, y1, color = 'indianred', alpha=1)
ax2.bar(x2, y2, color = 'seagreen', alpha=0.4)
  
# giving labels to the axises
ax.set_xlabel('Weeks', size=16)
ax.set_ylabel('Number of Unemployment Claims', size=16)
  
# secondary y-axis label
ax2.set_ylabel('Streams', size=16)
  
# defining display layout 
plt.tight_layout()
  
# show plot


fig.savefig('2020_Weekly_UIvsStreams.png', facecolor='w', dpi=600)
plt.show()

Create correlation and scatter plot for UI vs streams per week

In [ ]:
claims_ui=[]
all_wks_claims = new_ui_df['S.A.'].astype(float).to_list()
print(all_wks_claims)

In [ ]:
songs_week_list = weeks_reindex['Week Number'].to_list()
print(songs_week_list)

In [ ]:
print(len(week_num))

In [ ]:
week_claim_dict = dict(zip(week_num, all_wks_claims))
print(week_claim_dict)

In [ ]:
# to find only the claims number that match the same week in the songs
for x, y in week_claim_dict.items():
    n = 0
    for n in range(len(songs_week_list)):
        if x == songs_week_list[n]:
            claims_ui.append(int(y))
            break

In [ ]:
print(claims_ui)
print(len(claims_ui))
# print(claims_ui[0].type())
print(len(weeks_list))
print(num_streams)

In [ ]:
x = np.array([210000, 206000, 219000, 210000, 216000, 212000, 256000, 2923000, 2784000, 2315000, 2149000, 1887000, 1398000, 1479000, 1398000, 1262000, 881000, 860000, 860000, 803000, 728000, 732000, 762000, 719000])
y= np.array([568732447.0, 630848180.0, 699004731.0, 608490489.0, 632716011.0, 690261593.0, 590879939.0, 572606856.0, 567969812.0, 525600167.0, 532625341.0, 522477928.0, 512651814.0, 637517013.0, 541592640.0, 659279693.0, 510979316.0, 494926568.0, 493853348.0, 508319217.0, 482611166.0, 483252803.0, 514674608.0, 543690539.0])

(slope,intercept,r_value,pvalue,stderr)=st.linregress(x,y)
slope = int(slope)
regress_values=x*slope+intercept
line_eq="y = "+str(round(slope,2))+" x + "+str(round(intercept,2))

print(f'The line equation is {line_eq}')

fig, ax = plt.subplots(figsize=(10,5))
ax = plt.scatter(x,y, color='purple')
ax = plt.plot(x,regress_values,"indianred")
ax = plt.text(1000000,620000000, line_eq, size=14, color='b')
plt.xlabel('Number of Weekly Unemployment Claims', size=12)
plt.ylabel('Number of Total Streams per Week', size=12)
plt.tight_layout(pad=3.5)
plt.grid(True, alpha=0.5)
plt.title('Linear Regression for Weekly Unemployment Claims vs \nTotal Streams per Week in Samples 2020 Months', size=20)

print(f"The r-squared is: {r_value**2:.2f}")
plt.savefig('correlation.jpg', facecolor='w', edgcolor='w', dpi=600)
plt.show()